<a href="https://colab.research.google.com/github/ATIK2110018/geospatial-python/blob/main/kutubdia_lulc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio -q

In [ ]:
import numpy as np
import rasterio
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from rasterio.plot import show
import tempfile
import shutil

In [ ]:
def read_lulc_map(file_path):
    with rasterio.open(file_path) as src:
        lulc_map = src.read(1)
        return lulc_map, src.transform, src.crs

In [ ]:
lulc_1988, transform, crs = read_lulc_map('/content/drive/MyDrive/GIS/kutubdia_tif/Kutubdia_1988.tif')
lulc_1997, _, _ = read_lulc_map('/content/drive/MyDrive/GIS/kutubdia_tif/Kutubdia_1997.tif')
lulc_2006, _, _ = read_lulc_map('/content/drive/MyDrive/GIS/kutubdia_tif/Kutubdia_2006.tif')
lulc_2015, _, _ = read_lulc_map('/content/drive/MyDrive/GIS/kutubdia_tif/Kutubdia_2015tif.tif')
lulc_2024, _, _ = read_lulc_map('/content/drive/MyDrive/GIS/kutubdia_tif/Kutubdia_2024.tif')

In [ ]:
X = np.stack([lulc_1988.flatten(), lulc_1997.flatten(), lulc_2006.flatten(), lulc_2015.flatten()], axis=1)
y = lulc_2024.flatten()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

In [ ]:

import numpy as np
from sklearn.metrics import accuracy_score

lulc_2033_pred = rf_classifier.predict(np.stack([lulc_1997.flatten(), lulc_2006.flatten(), lulc_2015.flatten(), lulc_2024.flatten()], axis=1))
lulc_2042_pred = rf_classifier.predict(np.stack([lulc_2006.flatten(), lulc_2015.flatten(), lulc_2024.flatten(), lulc_2033_pred], axis=1))

lulc_2033_pred = lulc_2033_pred.reshape(lulc_1988.shape)
lulc_2042_pred = lulc_2042_pred.reshape(lulc_1988.shape)

y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

def save_raster(array, output_path, transform, crs):
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=array.shape[0],
        width=array.shape[1],
        count=1,
        dtype=array.dtype,
        crs=crs,
        transform=transform
    ) as dst:
        dst.write(array, 1)

temp_dir = tempfile.mkdtemp()
output_2033 = f"{temp_dir}/lulc_2033.tif"
output_2042 = f"{temp_dir}/lulc_2042.tif"

save_raster(lulc_2033_pred, output_2033, transform, crs)
save_raster(lulc_2042_pred, output_2042, transform, crs)

cell_size = 30  # Example cell size (replace with your actual cell size)
lulc_classes = {
    1: "Water",
    2: "Forest",
    3: "Agriculture",
    4: "Urban",
    5: "Mangrove" # Assign values as in the training maps
    }

def calculate_area(lulc_map):
  area_per_class = {}
  for class_value, class_name in lulc_classes.items():
    area_per_class[class_name] = np.sum(lulc_map == class_value) * cell_size * cell_size
  return area_per_class

area_2033 = calculate_area(lulc_2033_pred)
area_2042 = calculate_area(lulc_2042_pred)

print(f"Land area in 2033: {area_2033}")
print(f"Land area in 2042: {area_2042}")

!cp $output_2033 /content/drive/MyDrive/GIS/
!cp $output_2042 /content/drive/MyDrive/GIS/

shutil.rmtree(temp_dir)

In [ ]:

import matplotlib.pyplot as plt
show(lulc_2033, transform=transform)
plt.title('Predicted LULC Map for 2033')
plt.show()

show(lulc_2042, transform=transform)
plt.title('Predicted LULC Map for 2042')
plt.show()

In [ ]:


import matplotlib.pyplot as plt
from rasterio.plot import show
import rasterio

for year in [2033, 2042]:
    with rasterio.open(f"/content/lulc_{year}.tif") as src:
        lulc_data = src.read(1)
        show(lulc_data, transform=src.transform) # Use src.transform
        plt.title(f'Predicted LULC Map for {year}')
        plt.show()